In [ ]:
%matplotlib inline

import sys
sys.path.insert(0, "your lib")

# Please follow the official DINOv3 guidelines and configure the environment first.
# DINOv3: https://github.com/facebookresearch/dinov3

In [ ]:
# ==== Cell 1: Infrastructure and Global Configuration (Native Resolution Ready) ====

import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import numpy as np
import random, os, json, gc
from PIL import Image, ImageFilter, ImageOps
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm

# --- 1. Global Paths and Parameters ---
TRAIN_IMG_DIR   = "../../Dataset/Seepage/train"
VAL_IMG_DIR     = "../../Dataset/Seepage/val"
SUPPORT_JSON    = "./train1.json"
FULL_TRAIN_JSON = "../../Dataset/Seepage/train_simplified_571.json"
VAL_ANN_FILE    = "../../Dataset/Seepage/val_simplified_194.json"

# Model saving paths
TEACHER_PATH    = "./proto_fullres.npz"

# DINO Configuration
DINO_REPO       = "facebookresearch/dinov3" 
MODEL_NAME      = "dinov3_vitl16"
WEIGHTS_PATH    = "./dinov3_vitl16_pretrain.pth" 
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED            = 3407
PATCH_SIZE      = 16

def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(SEED)

# --- 2. DINOv3 Feature Extractor (Optimized Version) ---
class DINOv3Extractor:
    def __init__(self, repo, model_name, weights, device):
        self.device = device
        print(f"Loading {model_name}...")
        # Note: source='local' or 'github' depends on your local environment setup.
        # If GitHub loading is slow, consider cloning the repo and using source='local'.
        self.model = torch.hub.load(repo, model=model_name, source='github', pretrained=False)
        
        if os.path.exists(weights):
            checkpoint = torch.load(weights, map_location="cpu")
            sd = checkpoint["model"] if "model" in checkpoint else checkpoint
            # Weight cleaning logic to remove module/backbone prefixes
            clean_sd = {k.replace("module.", "").replace("backbone.", ""): v for k, v in sd.items()}
            self.model.load_state_dict(clean_sd, strict=False)
            print("✅ Weights loaded.")
        else:
            print(f"⚠️ Warning: Weights not found at {weights}")
            
        self.model.eval().to(device)

    def preprocess(self, img_pil, size):
        """
        Preprocessing: Resize (only if necessary) -> Tensor -> Normalize
        size: 
          - int: Forced square resize (e.g., 768) -> Used for specific testing
          - None: Native resolution mode (auto-align to patch size) -> Primary mode
        """
        w, h = img_pil.size

        if size is not None:
            # Forced mode
            img = TF.resize(img_pil, (size, size), interpolation=TF.InterpolationMode.BICUBIC)
        else:
            # Native mode: Resize only if dimensions are not multiples of 16
            # For 1600x1200, new_h=1200, new_w=1600; no resize will be triggered.
            new_h = (h // PATCH_SIZE) * PATCH_SIZE
            new_w = (w // PATCH_SIZE) * PATCH_SIZE
            
            if new_h != h or new_w != w:
                # Interpolate only when alignment is required
                img = TF.resize(img_pil, (new_h, new_w), interpolation=TF.InterpolationMode.BICUBIC)
            else:
                # Perfect dimensions: use original image to retain 100% sharpness
                img = img_pil
            
        img = TF.to_tensor(img)
        img = TF.normalize(img, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        return img.unsqueeze(0).to(self.device)

    @torch.no_grad()
    def get_patch_features(self, img_pil, size=None):
        """
        Extract patch-level features
        Returns: norm_patches [N, 1024], (H, W)
        """
        x = self.preprocess(img_pil, size)
        
        # Use get_intermediate_layers to retrieve features
        # n=1 indicates the last layer; reshape=True returns (B, C, H, W)
        out = self.model.get_intermediate_layers(x, n=1, reshape=True, norm=True)[0]
        
        B, D, H, W = out.shape
        # Flatten features: (H*W, D)
        patches = out.squeeze(0).permute(1, 2, 0).reshape(H*W, D)
        return F.normalize(patches, dim=1), (H, W)

# Initialization
if 'extractor' not in locals():
    extractor = DINOv3Extractor(DINO_REPO, MODEL_NAME, WEIGHTS_PATH, DEVICE)

# --- 3. Base Dataset Class (with orientation correction) ---
class BaseDataset:
    def __init__(self, img_dir, json_file):
        self.img_dir = img_dir
        self.coco = COCO(json_file)
        self.img_ids = self.coco.getImgIds()
        
    def get_image(self, idx):
        """Returns the PIL image and filename (automatically corrected for orientation)"""
        info = self.coco.loadImgs(self.img_ids[idx])[0]
        path = os.path.join(self.img_dir, info['file_name'])
        
        img = Image.open(path).convert("RGB")
        # Critical: Handle EXIF orientation tags to prevent vertical images from being read horizontally
        img = ImageOps.exif_transpose(img)
        
        return img, info['file_name']

print("Native Resolution Infrastructure Ready.")

In [ ]:
# Prototype part

In [ ]:
# ==== Cell 2: Native Multi-Scale Feature Mining ====
# Strategy:
# 1. Scale Augmentation: Enables model to perceive seepage textures at varying sizes.
# 2. Memory Safety: Automatic random cropping during upscaling to prevent OOM errors.
# 3. Grid Alignment: Forces dimensions to be multiples of 16 to reduce edge artifacts.

import torch
import torch.nn.functional as F
import numpy as np
import torchvision.transforms.functional as TF
from PIL import Image
import random

# --- 1. Parameters ---
DATA_JSON  = SUPPORT_JSON 
TARGET_ID  = 1   # Seepage
JOINT_ID   = 2   # Joint
STRUCT_IDS = [3, 4, 5]

# Increased rounds to account for added sample diversity from scaling
AUG_ROUNDS = 9 
SEED = 3407 
PATCH_SIZE = 16

def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

print(f"🔄 Resetting Seed to {SEED}...")
set_seed(SEED) 

# --- 2. Robust Multi-Scale Augmentation ---
def augment_pair_robust(img, mask):
    w, h = img.size
    
    # === A. Random Scaling ===
    # Range: 0.8 (Global view) ~ 1.5 (Detailed view)
    scale = random.uniform(0.8, 1.5)
    
    # Calculate new dimensions (align to Patch Size)
    new_h = int(h * scale)
    new_w = int(w * scale)
    new_h = (new_h // PATCH_SIZE) * PATCH_SIZE
    new_w = (new_w // PATCH_SIZE) * PATCH_SIZE
    
    # Size fallback
    if new_h < 512 or new_w < 512:
        new_h, new_w = h, w 
    
    # Resize (Bicubic for Image, Nearest for Mask)
    img = img.resize((new_w, new_h), Image.BICUBIC)
    mask = Image.fromarray(mask).resize((new_w, new_h), Image.NEAREST)
    
    # === B. Random Cropping (Memory Protection) ===
    # Trigger crop if pixels exceed 1600x1200 or based on probability
    max_pixels = 1600 * 1200
    curr_pixels = new_w * new_h
    
    if curr_pixels > max_pixels or random.random() > 0.3:
        # Define crop size (multiples of 16)
        crop_h = min(new_h, random.choice([768, 896, 1024]))
        crop_w = min(new_w, random.choice([768, 896, 1024]))
        
        if new_h > crop_h and new_w > crop_w:
            top = random.randint(0, new_h - crop_h)
            left = random.randint(0, new_w - crop_w)
            
            img = img.crop((left, top, left + crop_w, top + crop_h))
            mask = mask.crop((left, top, left + crop_w, top + crop_h))
    
    # === C. Geometric Transforms (Flip & Rotate) ===
    if random.random() > 0.5:
        img = TF.hflip(img); mask = TF.hflip(mask)
    if random.random() > 0.5:
        img = TF.vflip(img); mask = TF.vflip(mask)
    
    # 90-degree rotations
    if random.random() > 0.3:
        angle = random.choice([90, 180, 270])
        img = TF.rotate(img, angle)
        mask = TF.rotate(mask, angle)
        
    return img, np.array(mask)

# --- 3. Initialize Containers ---
feats_fg = []
feats_joint = []
feats_struct = []
feats_normal = [] 

# --- 4. Feature Extraction ---
print(f"🚀 Extracting Multi-Scale Features...")

if 'BaseDataset' not in globals(): raise RuntimeError("Please run Cell 1 first")
def get_len(self): return len(self.img_ids)
BaseDataset.__len__ = get_len
    
ds = BaseDataset(TRAIN_IMG_DIR, DATA_JSON)
model = extractor.model
model.eval()

for idx in range(len(ds)):
    img_orig, fname = ds.get_image(idx) # Orientation corrected
    
    # Parse GT Mask
    w, h = img_orig.size 
    mask_orig = np.zeros((h, w), dtype=np.uint8)
    ann_ids = ds.coco.getAnnIds(imgIds=ds.img_ids[idx])
    
    for ann in ds.coco.loadAnns(ann_ids):
        m = ds.coco.annToMask(ann)
        if m.shape != (h, w): # Size safety check
            m = np.array(Image.fromarray(m).resize((w, h), Image.NEAREST))
            
        cid = ann['category_id']
        if cid == TARGET_ID: mask_orig[m>0] = 1
        elif cid == JOINT_ID: mask_orig[m>0] = 2
        elif cid in STRUCT_IDS: mask_orig[m>0] = 3
    
    # 🔁 Augmentation Loop
    for r in range(AUG_ROUNDS + 1):
        if r == 0:
            # First round: Use original image (no scale/crop)
            curr_img, curr_mask = img_orig, mask_orig
        else:
            # Subsequent rounds: Retry mechanism to ensure key features are captured
            for _ in range(5): 
                res_img, res_mask = augment_pair_robust(img_orig, mask_orig)
                # Ensure either FG or Joint exists in the crop
                if 1 in res_mask or 2 in res_mask:
                    curr_img, curr_mask = res_img, res_mask
                    break
            else:
                # Skip round if key features aren't captured after 5 attempts
                continue
        
        # 1. Feature Extraction (size=None -> auto-adapt to current resolution)
        try:
            feat_map, (hf, wf) = extractor.get_patch_features(curr_img, size=None)
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"⚠️ OOM skipping round {r}")
                torch.cuda.empty_cache()
                continue
            continue
            
        feat_map = feat_map.cpu()

        # 3. Probability Map Generation
        # Note: (hf, wf) is dynamic across rounds
        def get_prob_map(val_id):
            mask_float = (curr_mask == val_id).astype(np.float32)
            t = torch.from_numpy(mask_float).float().unsqueeze(0).unsqueeze(0).to(DEVICE)
            # Bilinear interpolation for soft labels
            prob = F.interpolate(t, size=(hf, wf), mode='bilinear', align_corners=False).flatten()
            return prob.cpu()

        prob_fg = get_prob_map(1)
        prob_joint = get_prob_map(2)
        prob_struct = get_prob_map(3)
        
        # 4. Selection Logic
        # FG: High confidence seepage area
        idx_fg = (prob_fg > 0.6) & (prob_joint < 0.1) 
        
        # Joint
        idx_joint = (prob_joint > 0.4)
        
        # Struct
        idx_struct = (prob_struct > 0.6)
        
        # Normal (Background)
        idx_normal = (prob_fg < 0.05) & (prob_joint < 0.05) & (prob_struct < 0.05)
        
        # 5. Collection
        if idx_fg.any():     feats_fg.append(feat_map[idx_fg])
        if idx_joint.any():  feats_joint.append(feat_map[idx_joint])
        if idx_struct.any(): feats_struct.append(feat_map[idx_struct])
        
        # Background downsampling
        curr_norm = feat_map[idx_normal]
        if curr_norm.shape[0] > 3000:
            perm = torch.randperm(curr_norm.shape[0])[:3000]
            curr_norm = curr_norm[perm]
        if curr_norm.shape[0] > 0:
            feats_normal.append(curr_norm)

    print(f"   Processed {idx+1}/{len(ds)} images...")

print(f"✅ Multi-Scale Extraction Done!")

# Statistics
total_fg = sum(len(x) for x in feats_fg)
total_joint = sum(len(x) for x in feats_joint)
total_struct = sum(len(x) for x in feats_struct)
total_normal = sum(len(x) for x in feats_normal)

print(f"📊 Final Stats (Multi-Scale):")
print(f"   - FG Blocks:     {total_fg}")
print(f"   - Joint Blocks:  {total_joint}")
print(f"   - Struct Blocks: {total_struct}")
print(f"   - Normal Blocks: {total_normal}")

In [ ]:
# ==== Cell 3: Auto-Optimal K Search (Prototype Adaptation) ====
# Goal: Analyze feature space geometry to calculate the optimal number of cluster centers without inference.
# Method: Maximum Curvature (Kneedle Algorithm) / Elbow Method.
# Use Case: Handling massive feature sets under Native Resolution.

import torch
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# --- 1. Range Configuration ---
# In Native Mode, features are more diverse; ranges are widened accordingly.
# FG (Foreground): Seepage shapes vary (cracks, patches, spots), requiring sufficient K to cover.
FG_RANGE = [16, 32, 64, 80, 96, 128]

# BG (Background): Includes joints, structures, and concrete under various lighting—extremely high complexity.
BG_RANGE = [32, 64, 96, 128, 160, 192, 256, 320]

# Save path (automatically overwrites old prototype files)
SAVE_PATH = TEACHER_PATH

# --- 2. Core Algorithm: Mathematical "Inflection Point" Search ---
def find_optimal_k(feat_list, k_range, label_name):
    # 1. Data Preparation
    if not feat_list:
        print(f"⚠️ {label_name} is empty!")
        return 1, np.zeros((1, 1024))
    
    # Convert to Numpy (CPU)
    pool = torch.cat(feat_list, dim=0).numpy()
    N = pool.shape[0]
    
    # Filter invalid K (must be less than total samples)
    valid_k_range = [k for k in k_range if k < N]
    if not valid_k_range:
        valid_k_range = [N] # If samples are extremely few, K = sample count
    
    print(f"\n🔍 Analyzing {label_name} (Total Samples: {N})...")
    
    inertias = []
    models = {}
    
    # 2. Iteratively calculate Inertia (Within-cluster Sum of Squared errors)
    # Inertia decreases as K increases; we seek the point where the rate of decrease slows significantly.
    for k in tqdm(valid_k_range, leave=False, desc=f"Searching {label_name}"):
        kmeans = MiniBatchKMeans(
            n_clusters=k, 
            batch_size=4096, 
            n_init="auto", 
            random_state=3407
        ).fit(pool)
        
        inertias.append(kmeans.inertia_)
        models[k] = kmeans.cluster_centers_

    # 3. Mathematical "Elbow Point" Calculation
    # Normalize the curve to a [0, 1] square and calculate the distance from each point to the start-end line.
    x = np.array(range(len(inertias)))
    y = np.array(inertias)
    
    # Linear Normalization
    if len(x) > 1:
        x_norm = (x - x.min()) / (x.max() - x.min())
        y_norm = (y - y.min()) / (y.max() - y.min())
    else:
        return valid_k_range[0], models[valid_k_range[0]]
    
    # Vector geometry to calculate distance
    coords = np.vstack((x_norm, y_norm)).T
    p1, p2 = coords[0], coords[-1] # Start and End points
    vec = p2 - p1
    vec_norm = np.linalg.norm(vec)
    
    distances = []
    for p in coords:
        # Cross product to find height (distance) from the line segment
        # The point with maximum distance is the Elbow point.
        dist = np.abs(np.cross(vec, p - p1)) / (vec_norm + 1e-8)
        distances.append(dist)
        
    # Locate index with maximum distance
    best_idx = np.argmax(distances)
    best_k = valid_k_range[best_idx]
    
    # Print Analysis Table
    print(f"{'K':<5} | {'Inertia':<12} | {'Curvature':<10}")
    print("-" * 35)
    for i, k in enumerate(valid_k_range):
        mark = "👈 Best (Elbow)" if k == best_k else ""
        print(f"{k:<5} | {inertias[i]:<12.1f} | {distances[i]:<10.4f} {mark}")
        
    return best_k, models[best_k]

# --- 3. Execution and Saving ---

# Combine background features: Joint + Struct + Normal
all_bg_feats = feats_joint + feats_struct + feats_normal

print("🚀 Starting Automatic K Optimization...")

# A. Analyze Foreground (FG)
best_k_fg, centers_fg = find_optimal_k(feats_fg, FG_RANGE, "Foreground")

# B. Analyze Background (BG)
best_k_bg, centers_bg = find_optimal_k(all_bg_feats, BG_RANGE, "Background")

# C. Post-processing and Storage
print("\n" + "="*40)
print(f"🏆 Optimization Complete")
print(f"   FG Best K: {best_k_fg}")
print(f"   BG Best K: {best_k_bg}")

# Convert to Tensor and perform L2 Normalization (Mandatory for DINOv3)
p_fg_final = torch.from_numpy(centers_fg).float()
p_bg_final = torch.from_numpy(centers_bg).float()

p_fg_final = F.normalize(p_fg_final, dim=1)
p_bg_final = F.normalize(p_bg_final, dim=1)

# Save results
np.savez(SAVE_PATH, fg=p_fg_final.numpy(), bg=p_bg_final.numpy())

print(f"💾 Optimal Prototypes Saved to: {SAVE_PATH}")
print(f"   - FG Shape: {p_fg_final.shape}")
print(f"   - BG Shape: {p_bg_final.shape}")
print("="*40)

In [ ]:
# ==== Cell 3.5: Re-clustering ====

import torch
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import MiniBatchKMeans

# --- 1. Configuration ---
OVERRIDE_FG_K = best_k_fg 
OVERRIDE_BG_K = best_k_bg

SAVE_PATH = TEACHER_PATH

print(f"🔧 Manually Overriding Prototypes...")
print(f"   FG: {best_k_fg} -> {OVERRIDE_FG_K} (Boosting coverage)")
print(f"   BG: {best_k_bg} -> {OVERRIDE_BG_K} (Maintaining optimal)")

# --- 2. Re-clustering ---
def fast_cluster(feat_list, k):
    if not feat_list: return torch.zeros((0, 1024))
    pool = torch.cat(feat_list, dim=0).numpy()
    
    # Safety check: ensure K does not exceed total number of samples
    real_k = min(k, pool.shape[0])
    
    kmeans = MiniBatchKMeans(
        n_clusters=real_k, 
        batch_size=4096, 
        n_init="auto", 
        random_state=3407
    ).fit(pool)
    
    centers = torch.from_numpy(kmeans.cluster_centers_).float()
    return F.normalize(centers, dim=1)

# Re-generate prototypes
p_fg_new = fast_cluster(feats_fg, OVERRIDE_FG_K)
p_bg_new = fast_cluster(all_bg_feats, OVERRIDE_BG_K) # all_bg_feats inherited from previous cell

# --- 3. Save to Disk ---
np.savez(SAVE_PATH, fg=p_fg_new.numpy(), bg=p_bg_new.numpy())

print(f"\n✅ Overridden Prototypes Saved to: {SAVE_PATH}")
print(f"   - FG Shape: {p_fg_new.shape}")
print(f"   - BG Shape: {p_bg_new.shape}")

In [ ]:
# ==== Cell 4_TTA_Fixed_Base: Stable Target Resolution TTA (V13) ====

import os
# 🔥 Maintain VRAM configuration to prevent fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:64'

import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageOps
import time
import gc
import skimage.morphology as morph 
import random # For random rotation TTA

# ==========================================
# 1. Core Configuration
# ==========================================
PROTO_PATH  = "./proto_fullres.npz"
FINAL_TH    = 0.50  
PATCH_SIZE  = 16
# 🔥 Fixed TTA Target Base Resolution
BASE_H, BASE_W = 1200, 1600 

TTA_CONFIG = [
    # (scale, flip, rotation_degree)
    (1.0, False, 0),         # 1. Base 1.0x
    # (1.0, True, 0),          # 2. Flip 1.0x
    # (0.75, False, 0),        # 3. Base 0.75x
    # (0.75, True, 0),         # 4. Flip 0.75x
    # (1.0, False, 5),         # 5. Rotation +5 deg
    # (1.0, False, -5),        # 6. Rotation -5 deg
]

# Load Prototypes
pd = np.load(PROTO_PATH)
p_fg = torch.from_numpy(pd['fg']).float().to(DEVICE)
p_bg = torch.from_numpy(pd['bg']).float().to(DEVICE)
p_fg = F.normalize(p_fg, dim=1)
p_bg = F.normalize(p_bg, dim=1)

K_FG, K_BG = 32, 96

print(f"🚀 Starting Fixed Base TTA Inference (Base: {BASE_H}x{BASE_W}, Passes: {len(TTA_CONFIG)})...")

# ==========================================
# 2. Preprocessing & Environment Setup
# ==========================================
ds_val = BaseDataset(VAL_IMG_DIR, VAL_ANN_FILE)
extractor.model.eval()
torch.set_grad_enabled(False)

# ==========================================
# 3. Single Inference Function (Serial Processing for Stability)
# ==========================================
def infer_single_fixed_base(img_pil_orig, scale, flip, rotation_deg):
    w_orig, h_orig = img_pil_orig.size
    
    # 🔥 Fix: Target dimensions based on BASE_H/W and scale factor
    h_target_scaled = (int(BASE_H * scale) // PATCH_SIZE) * PATCH_SIZE
    w_target_scaled = (int(BASE_W * scale) // PATCH_SIZE) * PATCH_SIZE
    
    # Adaptive Portrait/Landscape orientation alignment
    if h_orig > w_orig:
        # Portrait (H > W): Force portrait target
        h_in, w_in = max(h_target_scaled, w_target_scaled), min(h_target_scaled, w_target_scaled)
    else:
        # Landscape (W > H) or Square: Force landscape target
        h_in, w_in = min(h_target_scaled, w_target_scaled), max(h_target_scaled, w_target_scaled)
        
    img_input = img_pil_orig
    
    # TTA Augmentations
    if flip: 
        img_input = ImageOps.mirror(img_input)
    if rotation_deg != 0:
        img_input = img_input.rotate(rotation_deg, resample=Image.BICUBIC, expand=False)
        
    img_tensor = None; out = None; prob_map = None
    
    try:
        # 1. Force Resize to calculated target dimensions
        img_input = img_input.resize((w_in, h_in), Image.BICUBIC)
        
        # 2. Preprocess
        img_tensor = extractor.preprocess(img_input, size=None)
        
        # 3. Forward Pass
        with torch.inference_mode():
            out = extractor.model.get_intermediate_layers(img_tensor, n=1, reshape=True, norm=True)[0]
            
            B, D, H_feat, W_feat = out.shape
            feat = out.squeeze(0).permute(1, 2, 0).reshape(-1, D)
            feat = F.normalize(feat, dim=1)
            
            # Similarity Matching
            sim_fg = torch.mm(feat, p_fg.t()); val_fg, _ = sim_fg.topk(K_FG, dim=1)
            sim_bg = torch.mm(feat, p_bg.t()); val_bg, _ = sim_bg.topk(K_BG, dim=1)
            
            logits = torch.cat([val_fg, val_bg], dim=1); probs = F.softmax(logits / 0.07, dim=1)
            score_fg = probs[:, :K_FG].sum(dim=1)
            
            score_map = score_fg.view(1, 1, H_feat, W_feat)
            
            # Interpolate back to original resolution
            prob_map = F.interpolate(score_map, size=(h_orig, w_orig), mode='bilinear', align_corners=False).squeeze()
            
            prob_map_np = prob_map.cpu().numpy()
            
            # 4. Reverse Augmentations (Rotation/Flip)
            if flip:
                prob_map_np = np.fliplr(prob_map_np)
            if rotation_deg != 0:
                # Rotate back using PIL (Nearest interpolation ensures pixel alignment)
                prob_map_np = np.array(Image.fromarray((prob_map_np * 255).astype(np.uint8)).rotate(-rotation_deg, resample=Image.NEAREST, expand=False))/255.0

            return prob_map_np

    except Exception as e:
        print(f"   ❌ Runtime Error at {scale}x scale: {e}")
        return None
    finally:
        # Clean up Python objects within TTA iteration
        del img_tensor, out, prob_map
        if 'feat' in locals(): del feat
        gc.collect() 


# ==========================================
# 4. Main Inference Loop
# ==========================================
pbar = tqdm(range(len(ds_val.img_ids)), desc="Fixed Base TTA", unit="img")
running_mious = []; running_fg_ious = []; results_dict = {} 
start_total_time = time.time()

for idx in pbar:
    img_pil, fname = ds_val.get_image(idx) 
    w_real, h_real = img_pil.size
    
    # Prepare Ground Truth (GT)
    gt = np.zeros((h_real, w_real), dtype=np.uint8)
    ann_ids = ds_val.coco.getAnnIds(imgIds=ds_val.img_ids[idx])
    for ann in ds_val.coco.loadAnns(ann_ids):
        if ann['category_id'] == 1: 
            m = ds_val.coco.annToMask(ann)
            if m.shape != (h_real, w_real): 
                m = np.array(Image.fromarray(m).resize((w_real, h_real), Image.NEAREST))
            gt = np.maximum(gt, m)

    # --- TTA Loop ---
    accum_prob_np = np.zeros((h_real, w_real), dtype=np.float32)
    valid_count = 0
    current_img_time = time.time()
    
    for scale, flip, rotation_deg in TTA_CONFIG:
        res = infer_single_fixed_base(img_pil, scale, flip, rotation_deg)
        if res is not None: 
            accum_prob_np += res
            valid_count += 1
            
    # --- Final Mask Generation & Metrics ---
    miou, fg_iou = 0.0, 0.0
    
    if valid_count > 0:
        p_final = accum_prob_np / valid_count
        final_mask = (p_final > FINAL_TH).astype(np.uint8)
        
        # Morphological Post-processing
        if final_mask.sum() > 0: 
            final_mask = morph.remove_small_objects(final_mask.astype(bool), min_size=50).astype(np.uint8)
            
        # IoU Computation
        i_area = ((final_mask==1)&(gt==1)).sum()
        u_area = ((final_mask==1)|(gt==1)).sum()
        curr_fg = i_area / (u_area + 1e-6)
        
        bg_i = ((final_mask==0)&(gt==0)).sum()
        bg_u = ((final_mask==0)|(gt==0)).sum()
        curr_miou = (curr_fg + bg_i/(bg_u+1e-6))/2.0
        
        miou = curr_miou; fg_iou = curr_fg
    
    # Store results
    results_dict[idx] = {'miou': miou, 'fg': fg_iou}
    running_mious.append(miou); running_fg_ious.append(fg_iou)
    
    # Progress Display Updates
    total_processed = idx + 1
    avg_total_time = (time.time() - start_total_time) / total_processed
    
    pbar.set_postfix({
        "mIoU": f"{np.mean(running_mious):.4f}", 
        "T/img": f"{time.time() - current_img_time:.2f}s",
        "Avg T": f"{avg_total_time:.2f}s"
    })

    # 🔥 Critical: Clear VRAM cache only when switching images
    del accum_prob_np, gt
    gc.collect() 
    torch.cuda.empty_cache() 

# ==========================================
# 5. Final Statistics
# ==========================================
total_time_taken = time.time() - start_total_time
print("\n" + "="*40)
print(f"🏆 Final Fixed Base TTA Results (Total {len(running_mious)} samples):")
print(f"   mIoU:   {np.mean(running_mious):.4f}")
print(f"   FG IoU: {np.mean(running_fg_ious):.4f}")
print(f"   Total Time: {total_time_taken/60:.2f} min ({total_time_taken:.2f} sec)")
print("="*40)

In [ ]:
# MLP part

In [ ]:
# ==== Cell 5: MLP-based Feature Extraction & Smart Mining ====
# 1. "Retry Mechanism": Ensures augmented crops contain seepage or joints to avoid wasted iterations.
# 2. VRAM Protection: Features are moved to CPU immediately after extraction.
# 3. Dimensional Alignment: Forced size=768 to ensure consistent DINOv3 patch ratios.

import torch
import torch.nn.functional as F
import numpy as np
import torchvision.transforms.functional as TF
from PIL import Image
import random

# --- 1. Configuration ---
DATA_JSON  = SUPPORT_JSON 
TARGET_ID  = 1   # Seepage
JOINT_ID   = 2   # Joint
STRUCT_IDS = [3, 4, 5]

AUG_ROUNDS = 9 
SEED = 3407 

# Reset Seed for Reproducibility
def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

print(f"🔄 Resetting Seed to {SEED}...")
set_seed(SEED) 

# --- 2. Augmentation Function ---
def augment_pair_multiclass(img, mask):
    # Random Cropping
    if random.random() > 0.3:
        w, h = img.size; scale = random.uniform(0.4, 1.0)
        nh, nw = int(h*scale), int(w*scale)
        top = random.randint(0, h-nh); left = random.randint(0, w-nw)
        img = TF.crop(img, top, left, nh, nw)
        mask = TF.crop(Image.fromarray(mask), top, left, nh, nw); mask = np.array(mask)
    # Flipping
    if random.random() > 0.5: img = TF.hflip(img); mask = np.fliplr(mask)
    if random.random() > 0.5: img = TF.vflip(img); mask = np.flipud(mask)
    # Rotation
    if random.random() > 0.125:
        angle = random.choice([45,90,135,180,225,270,315])
        img = TF.rotate(img, angle); mask = np.array(TF.rotate(Image.fromarray(mask), angle))
    return img, mask

# --- 3. Initialize Containers ---
feats_fg = []
feats_joint = []
feats_struct = []
feats_normal = [] 

# --- 4. Feature Extraction (Smart Mining Loop) ---
print(f"🚀 Extracting Features (Smart Mining Strategy)...")

# Dataset sanity check
if 'BaseDataset' not in globals(): raise RuntimeError("Please run Cell 1 first.")
def get_len(self): return len(self.img_ids)
BaseDataset.__len__ = get_len
    
ds = BaseDataset(TRAIN_IMG_DIR, DATA_JSON)
model = extractor.model
model.eval()

for idx in range(len(ds)):
    img_orig, fname = ds.get_image(idx)
    
    # Parse Ground Truth (GT) Mask
    w, h = img_orig.size; mask_orig = np.zeros((h, w), dtype=np.uint8)
    ann_ids = ds.coco.getAnnIds(imgIds=ds.img_ids[idx])
    for ann in ds.coco.loadAnns(ann_ids):
        m = ds.coco.annToMask(ann)
        cid = ann['category_id']
        if cid == TARGET_ID: mask_orig[m>0] = 1
        elif cid == JOINT_ID: mask_orig[m>0] = 2
        elif cid in STRUCT_IDS: mask_orig[m>0] = 3
    
    # 🔁 Augmentation Loop
    for r in range(AUG_ROUNDS + 1):
        # Strategy: Preserve original image in Round 0, use smart augmentation thereafter
        if r == 0:
            curr_img, curr_mask = img_orig, mask_orig
        else:
            # 🔥 Core Improvement: Smart Retry Mechanism
            # Attempt 3 times to crop an area containing Seepage (1) or Joints (2)
            # Proceed only if critical targets are found or retries are exhausted
            for _ in range(3):
                curr_img, curr_mask = augment_pair_multiclass(img_orig, mask_orig)
                if 1 in curr_mask or 2 in curr_mask: 
                    break
        
        # 1. Feature Extraction (Fixed size 768 for consistent Patch ratios)
        try:
            feat_map, (hf, wf) = extractor.get_patch_features(curr_img, size=768)
        except Exception as e:
            continue
            
        # 2. ⚡ Immediate Transfer to CPU (VRAM Protection)
        feat_map = feat_map.cpu()

        # 3. Probability Map Generation (Helper function)
        def get_prob_map(val_id):
            mask_float = (curr_mask == val_id).astype(np.float32)
            # Interpolation is faster on GPU
            t = torch.from_numpy(mask_float).float().unsqueeze(0).unsqueeze(0).to(DEVICE)
            prob = F.interpolate(t, size=(hf, wf), mode='bilinear', align_corners=False).flatten()
            return prob.cpu() # Transfer back to CPU after computation

        prob_fg = get_prob_map(1)
        prob_joint = get_prob_map(2)
        prob_struct = get_prob_map(3)
        
        # 4. Selection Logic (Boolean Indexing)
        # FG: High seepage probability, avoiding joint interference
        idx_fg = (prob_fg > 0.15) & (prob_joint < 0.10)
        
        # Joint: Confirmed joints (Hard negative samples)
        idx_joint = (prob_joint > 0.10)
        
        # Struct: Confirmed structural elements
        idx_struct = (prob_struct > 0.50)
        
        # Normal: Clean background
        idx_normal = (prob_fg < 0.05) & (prob_joint < 0.05) & (prob_struct < 0.05)
        
        # 5. Data Collection
        if idx_fg.any():     feats_fg.append(feat_map[idx_fg])
        if idx_joint.any():  feats_joint.append(feat_map[idx_joint])
        if idx_struct.any(): feats_struct.append(feat_map[idx_struct])
        
        # Normal Downsampling (Prevent memory overflow: max 2000 patches per image)
        curr_norm = feat_map[idx_normal]
        if curr_norm.shape[0] > 2000:
            perm = torch.randperm(curr_norm.shape[0])[:2000]
            curr_norm = curr_norm[perm]
        if curr_norm.shape[0] > 0:
            feats_normal.append(curr_norm)

    if (idx + 1) % 10 == 0:
        print

In [ ]:
# ==== Cell 6: MLP Training (Enforced Gradients + Balanced Augmentation) ====

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# --- 1. Configuration ---
MLP_SAVE_PATH = "./classifier_mlp_focal.pth"
SEED          = 3407
DEVICE        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE    = 4096
EPOCHS        = 150   
LR            = 0.0005
HIDDEN_DIM    = 256

# ⭐ Critical Fix 1: Force enable global gradients (prevents issues if a previous cell disabled them)
torch.set_grad_enabled(True)

torch.manual_seed(SEED)
np.random.seed(SEED)

# --- 2. Data Preparation ---
print(f"📥 Preparing Balanced Data...")
if 'feats_fg' not in locals(): raise RuntimeError("Please run Cell 6 first.")

def cat_feats(feat_list):
    if not feat_list: return torch.zeros((0, 1024))
    # Ensure conversion to float32 to prevent precision issues if DINO output is float16
    return torch.cat(feat_list, dim=0).float().cpu()

# Raw data concatenation
t_fg     = cat_feats(feats_fg)
t_joint  = cat_feats(feats_joint)
t_struct = cat_feats(feats_struct)
t_normal = cat_feats(feats_normal)

print(f"   [Raw] FG: {len(t_fg)}, Joint: {len(t_joint)}, Struct: {len(t_struct)}, Normal: {len(t_normal)}")

# 🔥 Strategy 1: Joint Oversampling
if len(t_joint) > 0:
    repeat_factor = len(t_fg) // len(t_joint)
    if repeat_factor > 1:
        print(f"   ⚡ Oversampling Joints by {repeat_factor}x ...")
        t_joint = t_joint.repeat(repeat_factor, 1)

# 🔥 Strategy 2: Background Downsampling
if len(t_normal) > 8000:
    perm = torch.randperm(len(t_normal))[:8000]
    t_normal = t_normal[perm]
    print(f"   ✂️ Downsampling Normal to {len(t_normal)}...")

# Merge datasets
X = torch.cat([t_fg, t_joint, t_struct, t_normal], dim=0)
# Labels: FG=1, Others=0
y = torch.cat([
    torch.ones(len(t_fg)), 
    torch.zeros(len(t_joint)),
    torch.zeros(len(t_struct)),
    torch.zeros(len(t_normal))
], dim=0).long()

print(f"   [Final] Training Set: {len(X)} samples.")

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# --- 3. Model Definition ---
class PatchClassifier(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=256):
        super(PatchClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(512, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_dim, 2)
        )
        
    def forward(self, x):
        return self.net(x)

# --- 4. Training Loop ---
print(f"🚀 Training Enhanced MLP...")

model = PatchClassifier(input_dim=1024, hidden_dim=HIDDEN_DIM).to(DEVICE)
model.train() # Ensure training mode

# Gradient check (debugging)
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"⚠️ Warning: {name} is frozen!")
        param.requires_grad = True

criterion = nn.CrossEntropyLoss(label_smoothing=0.1) 
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)



for epoch in range(EPOCHS):
    total_loss = 0
    correct = 0; total = 0
    
    for bx, by in loader:
        bx, by = bx.to(DEVICE), by.to(DEVICE)
        
        # 🔥 Strategy 3: Feature Noise Injection (Augmentation)
        if model.training:
            noise = torch.randn_like(bx) * 0.02
            bx = bx + noise
        
        optimizer.zero_grad()
        
        # Forward Pass
        logits = model(bx)
        
        # Loss Calculation
        loss = criterion(logits, by)
        
        # Backward Pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == by).sum().item()
        total += by.size(0)
        
    if (epoch+1) % 10 == 0:
        acc = correct / total
        print(f"   Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(loader):.5f} | Acc: {acc:.4f}")

torch.save(model.state_dict(), MLP_SAVE_PATH)
print(f"✅ Enhanced Model saved to: {MLP_SAVE_PATH}")

In [ ]:
# ==== Cell 7: Final Inference (Fixed Threshold 0.6 + TTA) ====

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from pycocotools.coco import COCO
from PIL import Image
import skimage.morphology as morph

# --- 1. Configuration ---
MLP_PATH       = "./classifier_mlp_focal.pth" # Must match training path in Cell 3
IMAGE_SIZE     = 768
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ⭐ Locked optimal threshold
FINAL_THRESHOLD = 0.60

# --- 2. Model Definition (Matching 3-layer MLP architecture) ---
print(f"🧠 Loading MLP from {MLP_PATH}...")

class PatchClassifier(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=256):
        super(PatchClassifier, self).__init__()
        # 3-layer MLP: 1024 -> 512 -> 256 -> 2
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(512, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_dim, 2)
        )
    def forward(self, x):
        return self.net(x)

# Instantiate and load weights
model = PatchClassifier(input_dim=1024, hidden_dim=256).to(DEVICE)

if not os.path.exists(MLP_PATH):
    raise FileNotFoundError(f"❌ Model file not found: {MLP_PATH}")

try:
    state_dict = torch.load(MLP_PATH, map_location=DEVICE, weights_only=True)
except:
    state_dict = torch.load(MLP_PATH, map_location=DEVICE, weights_only=False)

model.load_state_dict(state_dict)
model.eval()
print("✅ MLP Model Loaded Successfully.")

# --- 3. Dataset Preparation ---
if 'ValDataset' not in locals():
    class ValDataset:
        def __init__(self, img_dir, ann_file):
            self.img_dir = img_dir; self.coco = COCO(ann_file)
            self.img_ids = self.coco.getImgIds()
            self.data = [self.coco.loadImgs(i)[0] for i in self.img_ids if os.path.exists(os.path.join(img_dir, self.coco.loadImgs(i)[0]['file_name']))]
        def get_data(self, idx):
            info = self.data[idx]
            img = Image.open(os.path.join(self.img_dir, info['file_name'])).convert("RGB")
            mask = np.zeros((img.size[1], img.size[0]), dtype=np.uint8)
            for ann in self.coco.loadAnns(self.coco.getAnnIds(imgIds=info['id'])):
                if ann.get('category_id') == 1: mask = np.maximum(mask, self.coco.annToMask(ann))
            return img, mask, info['file_name']

val_dataset = ValDataset(VAL_IMG_DIR, VAL_ANN_FILE)

# --- 4. TTA Inference Functions ---
def predict_single(img_pil, extractor, mlp_model):
    w_orig, h_orig = img_pil.size
    img_t = extractor.preprocess(img_pil, IMAGE_SIZE)
    
    with torch.no_grad():
        # DINO Feature Extraction
        feat = extractor.model.get_intermediate_layers(img_t, n=1, reshape=True, norm=True)[0]
        
        # ✅ Normalize features for consistency
        feat = F.normalize(feat, dim=1)

        B, D, H, W = feat.shape
        # [1, 1024, H, W] -> [H*W, 1024]
        feat_flat = feat.permute(0, 2, 3, 1).reshape(-1, D)
        
        logits = mlp_model(feat_flat)
        prob = F.softmax(logits, dim=1)[:, 1].reshape(H, W)
        
        # Upsampling to original image resolution
        prob_high = F.interpolate(
            prob.unsqueeze(0).unsqueeze(0),
            size=(h_orig, w_orig), mode='bilinear', align_corners=False
        ).squeeze().cpu().numpy()
    return prob_high

def predict_with_tta(img_pil, extractor, mlp_model):
    # 1. Original Image Pass
    p1 = predict_single(img_pil, extractor, mlp_model)
    # 2. Horizontal Flip Pass (Test Time Augmentation)
    p2 = np.fliplr(predict_single(img_pil.transpose(Image.FLIP_LEFT_RIGHT), extractor, mlp_model))
    # 3. Ensemble Averaging
    return (p1 + p2) / 2.0

# --- 5. Execution (Applying 0.6 Threshold) ---
print(f"🚀 Running Inference (TTA + Threshold={FINAL_THRESHOLD})...")
results = []
ious = []
mious = []

pbar = tqdm(range(len(val_dataset.data)))
for idx in pbar:
    img, mask_np, fname = val_dataset.get_data(idx)
    gt = (mask_np > 0).astype(np.uint8)
    
    # Generate probability map
    score = predict_with_tta(img, extractor, model)
    
    # 1. Thresholding
    pred = (score > FINAL_THRESHOLD).astype(np.uint8)
    
    # 2. Morphological Post-processing (Denoising)
    if pred.sum() > 0:
        pred = morph.binary_opening(pred, morph.disk(1)).astype(np.uint8)
        pred = morph.remove_small_objects(pred.astype(bool), min_size=30).astype(np.uint8)
        
    # 3. Metrics Computation
    inter = ((pred==1) & (gt==1)).sum()
    union = ((pred==1) | (gt==1)).sum()
    iou = inter / (union + 1e-6)
    
    # mIoU (FG IoU + BG IoU) / 2
    inter_bg = ((pred==0) & (gt==0)).sum()
    union_bg = ((pred==0) | (gt==0)).sum()
    curr_miou = (iou + inter_bg/(union_bg+1e-6)) / 2.0
    
    ious.append(iou)
    mious.append(curr_miou)
    
    results.append({
        'name': fname, 'img': img, 'gt': gt, 
        'pred': pred, 'score': score, 'iou': iou
    })
    
    pbar.set_postfix({'FG_IoU': f"{np.mean(ious):.4f}"})

# --- 6. Final Statistics & Visualization ---
print(f"\n🏆 Final Results (Threshold={FINAL_THRESHOLD}):")
print(f"   FG IoU : {np.mean(ious):.4f}")
print(f"   mIoU   : {np.mean(mious):.4f}")

# Sort results by IoU for case analysis
results.sort(key=lambda x: x['iou'])

def plot_cases_final(cases, title):
    if not cases: return
    n = len(cases); plt.figure(figsize=(20, 4*n)); plt.suptitle(title, fontsize=20, y=1.005)
    for i, res in enumerate(cases):
        img = res['img']; gt = res['gt']; pred = res['pred']; score = res['score']
        h, w = gt.shape; vis = np.zeros((h, w, 3))
        
        # Color coding: Green=TP, Yellow=FN(Miss), Red=FP(False Alarm)
        vis[(gt==1) & (pred==1)] = [0, 1, 0] 
        vis[(gt==1) & (pred==0)] = [1, 1, 0] 
        vis[(gt==0) & (pred==1)] = [1, 0, 0] 
        
        img_np = np.array(img) / 255.0
        mask_any = (gt==1) | (pred==1)
        overlay = img_np.copy()
        if mask_any.sum() > 0: overlay[mask_any] = img_np[mask_any]*0.4 + vis[mask_any]*0.6
        
        plt.subplot(n, 4, i*4+1); plt.imshow(img); plt.title(f"{res['name']}\nIoU: {res['iou']:.3f}"); plt.axis('off')
        plt.subplot(n, 4, i*4+2); plt.imshow(gt, cmap='gray'); plt.title("GT"); plt.axis('off')
        plt.subplot(n, 4, i*4+3); plt.imshow(score, cmap='jet', vmin=0, vmax=1); plt.title(f"Score Max:{score.max():.2f}"); plt.axis('off')
        plt.subplot(n, 4, i*4+4); plt.imshow(overlay); plt.title("G=TP, Y=Miss, R=FP"); plt.axis('off')
    plt.tight_layout(); plt.show()


print("🔍 Analyzing Worst Cases")
plot_cases_final(results[:5], f"Worst Cases (Threshold={FINAL_THRESHOLD})")

In [ ]:
# Ensemble learning

In [ ]:
# ==== Cell 8: Final Inference (Fixed Threshold 0.6 + TTA) ====

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os, gc
from PIL import Image, ImageOps
import torchvision.transforms.functional as TF

# --- 1. Global Configuration ---
PROTO_PATH    = "./proto_fullres.npz"
MLP_PATH       = "./classifier_mlp_focal.pth"
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 2. Independent TTA (Test-Time Augmentation) Configurations ---

# [Branch 1: Prototype-based TTA Strategy]
# Based on the Fixed-Base logic from previous iterations
# Format: (scale, flip, rotation)
PROTO_TTA_CONFIG = [
    (1.0, False, 0),   # 1. Baseline
    (1.0, True, 0),    # 2. Horizontal Flip (Robustness)
    # (0.8, False, 0), # Optional: Downscale for global context
]
PROTO_BASE_SIZE = (1200, 1600) # (H, W) Base resolution

# [Branch 2: MLP-based TTA Strategy]
# Standard flip augmentation at fixed 768 resolution
MLP_TTA_OPS = ['orig', 'flip'] 

# --- 3. Model Definition ---
class PatchClassifier(nn.Module):
    """Simple MLP Classifier for feature patches."""
    def __init__(self, input_dim=1024, hidden_dim=256):
        super(PatchClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 2)
        )
    def forward(self, x):
        return self.net(x)

class HybridEnsemblePredictor:
    """Ensemble model combining Prototype similarity and MLP classification."""
    def __init__(self, extractor, proto_path, mlp_path, device):
        self.device = device
        self.extractor = extractor
        self.patch_size = 16
        
        # Load Prototypes (High-Res Branch)
        print(f"🔹 Loading Branch 1 (Proto High-Res)...")
        pd = np.load(proto_path)
        self.fg_p = torch.from_numpy(pd['fg']).float().to(device)
        self.bg_p = torch.from_numpy(pd['bg']).float().to(device)
        self.fg_p = F.normalize(self.fg_p, dim=1)
        self.bg_p = F.normalize(self.bg_p, dim=1)
        self.topk_fg = 64
        self.topk_bg = 96
        
        # Load MLP Classifier (Low-Res Branch)
        print(f"🔸 Loading Branch 2 (MLP Low-Res)...")
        self.mlp = PatchClassifier().to(device)
        try:
            state_dict = torch.load(mlp_path, map_location=device, weights_only=True)
        except:
            state_dict = torch.load(mlp_path, map_location=device)
        self.mlp.load_state_dict(state_dict)
        self.mlp.eval()
        self.extractor.model.eval()

    # ==================================================
    # Branch 1: Prototype Independent TTA (Fixed Base)
    # ==================================================
    def _infer_proto_single(self, img_pil, scale, flip, rotation):
        w_orig, h_orig = img_pil.size
        BASE_H, BASE_W = PROTO_BASE_SIZE
        
        # 1. Calculate target dimensions aligned with patch size
        h_target = (int(BASE_H * scale) // 16) * 16
        w_target = (int(BASE_W * scale) // 16) * 16
        
        # Adaptive Orientation (Portrait vs Landscape)
        if h_orig > w_orig:
            h_in, w_in = max(h_target, w_target), min(h_target, w_target)
        else:
            h_in, w_in = min(h_target, w_target), max(h_target, w_target)
            
        # 2. Apply TTA Transformations
        img_input = img_pil
        if flip: img_input = ImageOps.mirror(img_input)
        if rotation != 0: img_input = img_input.rotate(rotation, resample=Image.BICUBIC)
        
        # 3. Resize & Inference
        img_input = img_input.resize((w_in, h_in), Image.BICUBIC)
        img_t = self.extractor.preprocess(img_input, size=None) # Native mode processing
        
        with torch.no_grad():
            out = self.extractor.model.get_intermediate_layers(img_t, n=1, reshape=True, norm=True)[0]
            B, D, Hf, Wf = out.shape
            
            feat = F.normalize(out.squeeze(0).permute(1, 2, 0).reshape(-1, D), dim=1)
            
            # Compute similarity with prototypes
            sim_fg = torch.mm(feat, self.fg_p.t())
            sim_bg = torch.mm(feat, self.bg_p.t())
            
            val_fg, _ = sim_fg.topk(min(self.topk_fg, sim_fg.shape[1]), dim=1)
            val_bg, _ = sim_bg.topk(min(self.topk_bg, sim_bg.shape[1]), dim=1)
            
            logits = torch.cat([val_fg, val_bg], dim=1)
            probs = F.softmax(logits / 0.07, dim=1)
            
            score_fg = probs[:, :val_fg.shape[1]].sum(dim=1).view(1, 1, Hf, Wf)
            
            # 4. Upsample back to original image resolution
            prob_map = F.interpolate(score_fg, size=(h_orig, w_orig), mode='bilinear', align_corners=False).squeeze().cpu().numpy()
            
            # 5. Inverse TTA Transformations
            if rotation != 0:
                 prob_map = np.array(Image.fromarray((prob_map * 255).astype(np.uint8)).rotate(-rotation, resample=Image.NEAREST, expand=False))/255.0
            if flip:
                prob_map = np.fliplr(prob_map)
                
            return prob_map

    def _get_proto_tta_result(self, img_pil):
        """Aggregates TTA passes for the Prototype branch."""
        accum_map = np.zeros((img_pil.size[1], img_pil.size[0]), dtype=np.float32)
        count = 0
        for scale, flip, rot in PROTO_TTA_CONFIG:
            res = self._infer_proto_single(img_pil, scale, flip, rot)
            accum_map += res
            count += 1
        return accum_map / count

    # ==================================================
    # Branch 2: MLP Independent TTA (Fixed 768 Resolution)
    # ==================================================
    def _infer_mlp_single(self, img_pil, flip):
        w_orig, h_orig = img_pil.size
        # Apply TTA Flip
        if flip: img_pil = ImageOps.mirror(img_pil)
        
        # Forced 768 resolution for consistency
        img_t = self.extractor.preprocess(img_pil, size=768)
        
        with torch.no_grad():
            feat = self.extractor.model.get_intermediate_layers(img_t, n=1, reshape=True, norm=True)[0]
            feat = F.normalize(feat, dim=1)
            B, D, Hf, Wf = feat.shape
            
            logits = self.mlp(feat.permute(0, 2, 3, 1).reshape(-1, D))
            score_fg = F.softmax(logits, dim=1)[:, 1].view(1, 1, Hf, Wf)
            
            # Upsample
            prob_map = F.interpolate(score_fg, size=(h_orig, w_orig), mode='bilinear', align_corners=False).squeeze().cpu().numpy()
            
            # Inverse TTA Flip
            if flip: prob_map = np.fliplr(prob_map)
            return prob_map

    def _get_mlp_tta_result(self, img_pil):
        """Aggregates TTA passes for the MLP branch."""
        p1 = self._infer_mlp_single(img_pil, flip=False)
        p2 = self._infer_mlp_single(img_pil, flip=True)
        return (p1 + p2) / 2.0

    # ==================================================
    # Fusion Interface
    # ==================================================
    def predict_fusion_components(self, img_pil):
        """
        Executes TTA for both branches and returns two independent probability maps.
        Weights are not applied here to allow for external weight search loops.
        """
        p_proto = self._get_proto_tta_result(img_pil)
        p_mlp   = self._get_mlp_tta_result(img_pil)
        return p_proto, p_mlp

# Initialization
if 'extractor' in locals():
    ensemble_model = HybridEnsemblePredictor(extractor, PROTO_PATH, MLP_PATH, DEVICE)
    print("✅ Model initialized with Independent Dual-TTA.")
else:
    print("❌ Please load the 'extractor' first (Infrastructure Cell).")

In [ ]:
# ==== Cell9: Alpha & Threshold Joint Grid Search ====

import numpy as np
import skimage.morphology as morph
from tqdm import tqdm

# --- 1. Define Search Space ---
SEARCH_ALPHAS = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# 🔥 Key Improvement: Thresholds are no longer fixed; testing three critical points dynamically
SEARCH_THRESHOLDS = [0.50, 0.55, 0.60] 

# Morphological Post-processing (Settings proven effective for MLP)
USE_OPENING = True 
MIN_SIZE    = 30

print(f"🚀 Starting Joint Search (Alpha x Threshold)...")
print(f"   Goal: Identify optimal scores across the entire ensemble spectrum.")

# Ensure cache_results exists (Run the previous caching cell if missing)
if 'cache_results' not in locals() or len(cache_results) == 0:
    raise RuntimeError("❌ 'cache_results' not found! Please run the caching cell first.")

best_global_miou = 0
best_global_alpha = 0
best_global_th = 0

print(f"{'Alpha':<6} | {'Best Th':<8} | {'mIoU':<8} | {'FG IoU':<8} | {'Notes'}")
print("-" * 60)

for alpha in SEARCH_ALPHAS:
    # Track the best performance for the current Alpha
    current_alpha_best_miou = 0
    current_alpha_best_th   = 0
    current_alpha_fg        = 0
    
    # --- Inner Loop: Finding the best threshold for the current mixture ratio ---
    for th in SEARCH_THRESHOLDS:
        mious = []
        fg_ious = []
        
        for item in cache_results:
            # 1. Linear Fusion (Weighted Averaging)
            score = alpha * item['p_proto'] + (1 - alpha) * item['p_mlp']
            
            # 2. Dynamic Thresholding
            pred = (score > th).astype(np.uint8)
            
            # 3. Post-processing (Opening + Small Object Removal)
            if pred.sum() > 0:
                if USE_OPENING:
                    pred = morph.binary_opening(pred, morph.disk(1)).astype(np.uint8)
                pred = morph.remove_small_objects(pred.astype(bool), min_size=MIN_SIZE).astype(np.uint8)
            
            # 4. Metric Calculation
            gt = item['gt']
            i = ((pred==1) & (gt==1)).sum()
            u = ((pred==1) | (gt==1)).sum()
            iou = i / (u + 1e-6)
            
            bg_i = ((pred==0) & (gt==0)).sum()
            bg_u = ((pred==0) | (gt==0)).sum()
            miou = (iou + bg_i/(bg_u+1e-6)) / 2.0
            
            mious.append(miou)
            fg_ious.append(iou)
        
        # Check if this threshold is the best for the current Alpha
        avg_miou = np.mean(mious)
        if avg_miou > current_alpha_best_miou:
            current_alpha_best_miou = avg_miou
            current_alpha_best_th   = th
            current_alpha_fg        = np.mean(fg_ious)

    # Print the best result for the current Alpha
    # Marker: Star indicates a new global high for mIoU
    marker = "⭐" if current_alpha_best_miou > best_global_miou else ""
    
    print(f"{alpha:<6} | {current_alpha_best_th:<8} | {current_alpha_best_miou:.4f}   | {current_alpha_fg:.4f}   | {marker}")
    
    # Update Global Optimum
    if current_alpha_best_miou > best_global_miou:
        best_global_miou  = current_alpha_best_miou
        best_global_alpha = alpha
        best_global_th    = current_alpha_best_th

print("\n" + "="*50)
print(f"🏆 GLOBAL BEST CONFIGURATION:")
print(f"   Alpha     : {best_global_alpha}")
print(f"   Threshold : {best_global_th}")
print(f"   Max mIoU  : {best_global_miou:.4f}")
print("="*50)

In [ ]:
# ==== Cell 10: GRAND FINAL PREDICTION (TTA + Alpha 0.6 + Th 0.55 + CRF) ====
# Dependency: !pip install pydensecrf

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax
from PIL import Image
import skimage.morphology as morph

# ==========================================
# 1. Champion Configuration (Derived from Joint Search)
# ==========================================
BEST_ALPHA     = 0.60   # 🏆 Optimal Weight (60% Proto / 40% MLP)
FINAL_TH       = 0.55   # 🏆 Optimal Threshold (Balanced point)
USE_CRF        = True   # Enable DenseCRF for boundary refinement

# Morphological Post-processing (Consistent with search phase)
USE_OPENING    = True
MIN_SIZE       = 30

# DenseCRF Parameters (High-precision configuration)
CRF_ITER       = 5
CRF_POS_W      = 3; CRF_POS_XY  = 3
CRF_Bi_W       = 5; CRF_Bi_XY   = 50; CRF_Bi_RGB  = 10

print(f"🚀 Starting Grand Final Inference...")
print(f"   Config: Alpha={BEST_ALPHA} | Th={FINAL_TH} | CRF=ON")

# ==========================================
# 2. Core CRF Function
# ==========================================
def apply_dense_crf(img_np, prob_map):
    h, w = prob_map.shape
    # Stack probabilities for background and foreground
    prob_stack = np.stack([1 - prob_map, prob_map], axis=0)
    prob_stack = np.clip(prob_stack, 1e-6, 1.0)
    
    d = dcrf.DenseCRF2D(w, h, 2)
    U = unary_from_softmax(prob_stack)
    d.setUnaryEnergy(U)

    # 1. Pairwise Gaussian Potential (Smoothness)
    d.addPairwiseGaussian(sxy=(CRF_POS_XY, CRF_POS_XY), compat=CRF_POS_W, 
                          kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    # 2. Pairwise Bilateral Potential (Appearance-based refinement)
    d.addPairwiseBilateral(sxy=(CRF_Bi_XY, CRF_Bi_XY), srgb=(CRF_Bi_RGB, CRF_Bi_RGB, CRF_Bi_RGB), 
                           rgbim=img_np, compat=CRF_Bi_W, 
                           kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    Q = d.inference(CRF_ITER)
    return np.argmax(Q, axis=0).reshape((h, w)).astype(np.uint8)

# ==========================================
# 3. Execution of Final Inference
# ==========================================
if 'ds_search' not in locals():
    ds_final = SearchDataset(VAL_IMG_DIR, VAL_ANN_FILE)
else:
    ds_final = ds_search

results_final = []
ious_base = []
ious_crf = []
mious_crf = []

pbar = tqdm(range(len(ds_final.img_ids)))

for idx in pbar:
    img_pil, gt = ds_final.get_data(idx)
    img_np = np.array(img_pil)
    
    # 1. Get Dual-Branch TTA Probabilities
    p_proto, p_mlp = ensemble_model.predict_components(img_pil)
    
    # 2. Fusion (Alpha=0.6)
    score_map = BEST_ALPHA * p_proto + (1 - BEST_ALPHA) * p_mlp
    
    # --- A. Base Prediction (Ensemble Only) ---
    pred_base = (score_map > FINAL_TH).astype(np.uint8)
    
    # Apply morphological cleanup
    if pred_base.sum() > 0:
        if USE_OPENING:
            pred_base = morph.binary_opening(pred_base, morph.disk(1)).astype(np.uint8)
        pred_base = morph.remove_small_objects(pred_base.astype(bool), min_size=MIN_SIZE).astype(np.uint8)
    
    # Base Metric
    i_b = ((pred_base==1) & (gt==1)).sum()
    u_b = ((pred_base==1) | (gt==1)).sum()
    iou_base_val = i_b / (u_b + 1e-6)
    ious_base.append(iou_base_val)

    # --- B. CRF Prediction (Refinement) ---
    if USE_CRF:
        # CRF operates directly on the probability map (threshold-free)
        pred_crf = apply_dense_crf(img_np, score_map)
        # Safety cleanup for CRF artifacts
        if pred_crf.sum() > 0:
            pred_crf = morph.remove_small_objects(pred_crf.astype(bool), min_size=30).astype(np.uint8)
    else:
        pred_crf = pred_base
        
    # CRF Metric
    i_c = ((pred_crf==1) & (gt==1)).sum()
    u_c = ((pred_crf==1) | (gt==1)).sum()
    curr_iou_crf = i_c / (u_c + 1e-6)
    
    bg_i = ((pred_crf==0) & (gt==0)).sum()
    bg_u = ((pred_crf==0) | (gt==0)).sum()
    curr_miou_crf = (curr_iou_crf + bg_i/(bg_u+1e-6)) / 2.0
    
    ious_crf.append(curr_iou_crf)
    mious_crf.append(curr_miou_crf)
    
    # Collect data for visualization (Sample bad cases or periodic samples)
    if idx % 20 == 0 or curr_iou_crf < 0.6: 
        results_final.append({
            'img': img_np, 'gt': gt, 'score': score_map,
            'pred_base': pred_base, 'pred_crf': pred_crf,
            'iou_base': iou_base_val, 'iou_crf': curr_iou_crf
        })
        
    pbar.set_postfix({
        'Base': f"{np.mean(ious_base):.4f}", 
        'CRF': f"{np.mean(ious_crf):.4f}"
    })

# ==========================================
# 4. Final Performance Report
# ==========================================
print("\n" + "="*50)
print(f"🏆 FINAL PERFORMANCE REPORT")
print(f"   Ensemble   : Alpha={BEST_ALPHA}, Th={FINAL_TH}")
print(f"   Morphology : Opening=True, MinSize={MIN_SIZE}")
print(f"   CRF        : Enabled (Iter={CRF_ITER})")
print(f"   ----------------------------------------")
print(f"   Base FG IoU : {np.mean(ious_base):.4f}")
print(f"   Base mIoU   : {(np.mean(ious_base) + 0.9590)/2:.4f} (Est.)") 
print(f"   ----------------------------------------")
print(f"   CRF  FG IoU : {np.mean(ious_crf):.4f}")
print(f"   CRF  mIoU   : {np.mean(mious_crf):.4f}  <-- OFFICIAL FINAL SCORE")
print("="*50)

# ==========================================
# 5. Visualization of Top Improvements
# ==========================================

# 1. Calculate improvements
for res in results_final:
    res['diff'] = res['iou_crf'] - res['iou_base']

# 2. Sort by improvement gain
top_gains = sorted(results_final, key=lambda x: x['diff'], reverse=True)[:3]

def plot_final_comparison(cases, title):
    if not cases: return
    n = len(cases)
    plt.figure(figsize=(18, 5*n))
    plt.suptitle(title, fontsize=20, y=1.01)
    
    for i, res in enumerate(cases):
        # 1. Original Image
        plt.subplot(n, 4, i*4+1)
        plt.imshow(res['img'])
        plt.title(f"Original Image", fontsize=12); plt.axis('off')
        
        # 2. Ground Truth
        plt.subplot(n, 4, i*4+2)
        plt.imshow(res['gt'], cmap='gray')
        plt.title("Ground Truth", fontsize=12); plt.axis('off')
        
        # 3. Base Prediction
        plt.subplot(n, 4, i*4+3)
        plt.imshow(res['pred_base'], cmap='gray')
        plt.title(f"Ensemble Only\nIoU: {res['iou_base']:.3f}", fontsize=12); plt.axis('off')
        
        # 4. CRF Prediction
        plt.subplot(n, 4, i*4+4)
        plt.imshow(res['pred_crf'], cmap='gray')
        plt.title(f"Ensemble + CRF\nIoU: {res['iou_crf']:.3f}\n(Gain: +{res['diff']:.3f})", fontsize=12); plt.axis('off')
        
    plt.tight_layout()
    plt.show()

print(f"🌟 Final Score Analysis:")
print(f"   mIoU Reached: {np.mean(mious_crf):.4f}")
print(f"   FG IoU Reached: {np.mean(ious_crf):.4f}")
print("-" * 30)
print("🔍 Visualizing Top 3 CRF Improvements (Where CRF saved the day)...")
plot_final_comparison(top_gains, "Top 3 CRF Improvements")